In [1]:
import itertools # 是 Python 的內建模組，提供了一組用於處理迭代器的函數和工具。
                 # 它包含了各種用於高效處理迭代器的函數，可以幫助我們編寫更簡潔、高效的代碼。
import sys # 是 Python 的內建模組，提供了與 Python 解釋器和運行環境相關的功能。
# sys.path 是一個列表，包含了 Python 解釋器在導入模組時會搜尋的路徑。
# 當你使用 import 語句導入模組時 Python 會依次在 sys.path 中的路徑下尋找對應的模組文件。
sys.path.append("../ecg-classification/")
# sys.path.append("C:\\Users\\Chen_Lab01\\Documents\\GitHub/ecg-classification")
# from IPython.display import Video
# import cv2
from matplotlib import pyplot as plt
plt.style.use("ggplot") #  是 Matplotlib 庫中用於設置繪圖樣式的函數。它使用了一種名為 "ggplot" 的預定義樣式
import matplotlib
matplotlib.use("TkAgg")
                        #  該樣式模仿了 R 語言的 ggplot2 繪圖包的外觀。
# print(sys.path)
import torch
from ecg_tools.config import EcgConfig, Mode
from ecg_tools.data_loader import DatasetConfig, get_data_loaders
from ecg_tools.model import ECGformer
from ecg_tools.train import ECGClassifierTrainer


# Visualize the model

## Load

Load the model

In [2]:
model = torch.load("..\\..\\model_save\\model_epoch_598.pth")
model.eval()
model.to('cpu')

ECGformer(
  (encoder): ModuleList(
    (0): TransformerEncoderLayer(
      (0): ResidualAdd(
        (block): Sequential(
          (0): MultiHeadAttention(
            (queries_projection): Linear(in_features=64, out_features=64, bias=True)
            (values_projection): Linear(in_features=64, out_features=64, bias=True)
            (keys_projection): Linear(in_features=64, out_features=64, bias=True)
            (final_projection): Linear(in_features=64, out_features=64, bias=True)
          )
          (1): Dropout(p=0.1, inplace=False)
        )
      )
    )
  )
  (classifier): Classifier(
    (0): Reduce('b n e -> b e', 'mean')
    (1): Linear(in_features=64, out_features=2, bias=True)
  )
  (embedding): LinearEmbedding(
    (0): Linear(in_features=1, out_features=64, bias=True)
    (1): GELU(approximate='none')
  )
)

Load quantized model

In [3]:
config = EcgConfig()    
model_quantized = torch.load("..\\..\\model_save\\model_quantized_598.pth")
model_quantized.eval()
model_quantized.to('cpu')

C:\Users\Xaio\anaconda3\envs\pytorch-ecg\lib\site-packages\torch\_utils.py:382: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


ECGformer(
  (encoder): ModuleList(
    (0): TransformerEncoderLayer(
      (0): ResidualAdd(
        (block): Sequential(
          (0): MultiHeadAttention(
            (queries_projection): DynamicQuantizedLinear(in_features=64, out_features=64, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (values_projection): DynamicQuantizedLinear(in_features=64, out_features=64, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (keys_projection): DynamicQuantizedLinear(in_features=64, out_features=64, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (final_projection): DynamicQuantizedLinear(in_features=64, out_features=64, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          )
          (1): Dropout(p=0.1, inplace=False)
        )
      )
    )
  )
  (classifier): Classifier(
    (0): Reduce('b n e -> b e', 'mean')
    (1): DynamicQuantizedLinear(in_features=64, out_features=2, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  )
  (embe

print model

## TorchInfo summary

In [4]:
print(model_quantized)

ECGformer(
  (encoder): ModuleList(
    (0): TransformerEncoderLayer(
      (0): ResidualAdd(
        (block): Sequential(
          (0): MultiHeadAttention(
            (queries_projection): DynamicQuantizedLinear(in_features=64, out_features=64, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (values_projection): DynamicQuantizedLinear(in_features=64, out_features=64, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (keys_projection): DynamicQuantizedLinear(in_features=64, out_features=64, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (final_projection): DynamicQuantizedLinear(in_features=64, out_features=64, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          )
          (1): Dropout(p=0.1, inplace=False)
        )
      )
    )
  )
  (classifier): Classifier(
    (0): Reduce('b n e -> b e', 'mean')
    (1): DynamicQuantizedLinear(in_features=64, out_features=2, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  )
  (embe

model summary

In [ ]:
from torchinfo import summary
import sys

# Save reference to original stdout
original_stdout = sys.stdout

# Redirect stdout to file and call summary()
with open('model-summary.txt', 'w') as f:
    sys.stdout = f
    print(f"Model Summary for {type(model_quantized).__name__}:")
    summary(model_quantized, 
            input_size=(1, config.model.signal_length, 1),
            device='cpu', 
            col_names=("input_size", "output_size"),
            depth=6,
            verbose=1)

# Reset stdout back to original
sys.stdout = original_stdout

print("Model summary saved to model-summary.txt")


## Neutron 可視化

### ONNX 檔案
Neutron 可視化最齊全的是 ONNX 格式，但是不支援 quantized model，這裡使用原始 model

In [6]:
input_names = ["ECG Classification Input"]
output_names = ["Prediction"]
X = torch.randn(1, 63, 1)
torch.onnx.export(model, X, "model.onnx", input_names=input_names, output_names=output_names)

### TorchScript 檔案

In [7]:
inputs = torch.randn(1, 63, 1)
traced_script_module = torch.jit.trace(model_quantized, inputs)
traced_script_module.save("traced_resnet_model.pth")

import netron
modelData = 'traced_resnet_model.pth'
netron.start(modelData)

Serving 'traced_resnet_model.pth' at http://localhost:8080


('localhost', 8080)

## Grapviz 可視化

In [ ]:
from torchviz import make_dot
vis_graph = make_dot(model_quantized(inputs), params=dict(model_quantized.named_parameters()), show_attrs=True, show_saved=True)
vis_graph.view()  # 當前目錄保存為 pdf 文件